# Introdução

Este notebook apresenta um conjunto de códigos desenvolvidos para o pré-processamento e organização da banco de imagens utilizado.

Tópicos abordados:

* **Detecção facial:** aplicação do modelo RetinaFace para extração das faces das imagens;
* **Leave-some-subjects-out:** protocolo de divisão de dados em treinamento e teste, considerando um processo de validação cruzada;
* **Data Augmentation:** criação de novas amostras para aumento do conjunto de dados para treinamento. As novas amostras são validadas pelo RetinaFace para garantir que a face humana não foi descaracterizada;
* **Divisão "Dor" e "Sem_dor"**: rotina desenvolvida para separar as imagens de cada indivíduo nas classes mencionadas.

# Configuração

### Bibliotecas

In [ ]:
from shutil import copyfile, copytree, move
from tqdm import tqdm
import cv2
import os
import os.path as osp
import pandas as pd
import numpy as np

def log(texto,titulo):

    #print(texto)
    with open(titulo+'.txt',"a") as txt:
        txt.write(texto + '\n')

### Instalação do RetinaFace

O RetinaFace é o detector de faces proposto por DENG et al. (2020), no trabalho "RetinaFace: Single-Shot Multi-Level Face Localisation in the Wild" (disponível em https://arxiv.org/abs/1905.00641)

O modelo é encontrado no github do projeto InsightFace: https://github.com/deepinsight/insightface/tree/master/detection/retinaface

In [ ]:
## Download do modelo

# a versão do RetinaFace utilizada está no Google Drive do projeto InsightFace,
# portanto, utilizou-se a biblioteca gdown para o seu download
import gdown

original_url = 'https://drive.google.com/file/d/1wm-6K688HQEx_H90UdAIuKv-NAsKBu85/view?usp=sharing'

# url ajustada para a função do gdown (substitui) 
url_id = original_url.split('/d/')[-1].split('/')[0]
url = 'https://drive.google.com/uc?id='+url_id

# nome do arquivo a ser salvo
output = 'retinaface-R50.zip'

#download
gdown.download(url, output, quiet=False)

#instalação das bibliotecas
!pip install mxnet --quiet
!pip install insightface==0.1.5 --quiet

#criando o diretório para o modelo encontrado no Google Drive
!mkdir /root/.insightface
!mkdir /root/.insightface/models
!mkdir /root/.insightface/models/retinaface_r50_v1

#descompactando arquivo do modelo
!unzip -q retinaface-R50.zip -d /root/.insightface/models/retinaface_r50_v1/

#AJUSTE DOS ARQUIVOS DO PACKAGE
package_path = '/usr/local/lib/python3.7/dist-packages/insightface/model_zoo'

import os
import os.path as osp

txt=""
#--Ajustando arquivo model_zoo.py
fname='model_zoo.py'
with open(osp.join(package_path,fname),'rt') as f:
  txt = f.read().replace("from .face","from insightface.model_zoo.face")

with open(osp.join(package_path,fname),'wt') as f:
  f.write(txt)

#--Ajustando arquivo face_detection.py
fname='face_detection.py'
with open(osp.join(package_path,fname),'rt') as f:
  txt = f.read().replace("network=='net5'","self.rac=='net5'").replace("from .model_store","from insightface.model_zoo.model_store")

with open(osp.join(package_path,fname),'wt') as f:
  f.write(txt)

#--Ajustando arquivo model_store.py
fname='model_store.py'
with open(osp.join(package_path,fname),'rt') as f:
  txt = f.read().replace("from ..utils import download, check_sha1","from insightface.utils.download import *") 

with open(osp.join(package_path,fname),'wt') as f:
  f.write(txt)

#Instanciando um modelo Retinaface
import insightface
retinaface = insightface.model_zoo.get_model('retinaface_r50_v1')
retinaface.prepare(ctx_id = -1) 


#funcao de deteccao
def face_detection(image_path,model):

    img = cv2.imread(image_path)
    img = cv2.resize(img,(600,600))   


    box, landmarks = model.detect(img)
    box = box[0]

    x = abs(int(box[0]))
    y = abs(int(box[1]))
    w = abs(int(box[2])) - x
    h = abs(int(box[3])) - y

    face = img[y:y+h,x:x+w]
    new_landmarks=[]
    for a,b in landmarks[0]:
        x_new = a - x
        y_new = b - y 
        new_landmarks.append([x_new,y_new])

    return face, np.array(new_landmarks,dtype='float32')




# Detecção de Faces

As faces devem se encontrar na posição vertical antes da aplicação do RetinaFace

In [ ]:
#Detecção Faces

path = 'diretorio das imagens originais'
path_new = 'diretorio no qual as imagens de face serão gravadas'


c_no_faces=0
for imagem in os.listdir(path):
  
  try:
    face, _ = face_detection(os.path.join(path,imagem),retinaface)
    cv2.imwrite(os.path.join(path_new,imagem),face)
        
  except Exception:
    print('Nao foi possivel detectar faces na imagem {}'.format(imagem))
    c_no_faces+=1
    pass
    
c_total = len(os.listdir(path))

log("Total de imagens na base: "+str(c_total),log_name)
log("Total de imagens perdidas: "+str(c_no_faces),log_name)

# Leave-some-subjects-out

Nesta seção, será apresentada o método de divisão dos dados denominado *leave-some-subjects-out*. Trata-se do método de validação cruzada *k-fold* aplicado aos indivíduos (*subjects*) da base de dados, e não diretamente às amostras.

O trabalho em questão utilizou a união de duas bases de dados distintas.
Antes da execução do script a seguir, as imagens de cada base foram organizadas em pastas de acordo com o indivíduo da amostra.

Por exemplo:

*   Base A
  *   ID_01
      * ID01_imagem_1
      * ID01_imagem_2
      * ...
      * ID01_imagem_*N*

  * ID_02
  * ID_03
  * ...
  * ID_*N*

*   Base B


In [ ]:
cope_path='diretorio da base iCOPE'
unifesp_path='diretorio da base UNIFESP'

# Subjects da base iCOPE
cope_nbs = os.listdir(cope_path)
# quantidade de subjects iCOPE
print("Subjects iCOPE: ",len(cope_nbs))

# Subjects da base UNIFESP
unifesp_nbs = os.listdir(unifesp_path)
# quantidade de subjects UNIFESP
print("Subjects UNIFESP: ",len(unifesp_nbs))

Subjects iCOPE:  26
Subjects UNIFESP:  30


In [ ]:
# criação de uma lista com os "rótulos" dos subjects ('C' p/ iCOPE e 'U' p/ UNIFESP)
labels = len(cope_nbs)*['C'] + len(unifesp_nbs)*['U']

# 26 Cs + 30 Us = 56 rótulos
print("Total de rótulos: ",len(labels))

Total de rótulos:  56


In [ ]:
# lista com todos os subjects
nbs = cope_nbs + unifesp_nbs

# 26 subjects iCOPE + 30 subjects UNIFESP = 56 subjects
print("Total de subjects: ",len(nbs))

Total de subjects:  56


A função *StratifiedKFold* do módulo *sklearn* realiza a divisão *k-fold* balanceando dentro de cada *fold*. Em outras palavras, os *folds* possuem aproximadamente a mesma quantidade de imagens de cada classe. No contexto deste trabalho, a classe corresponde à base de dados de origem do *subject*.

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
# configurando o StratifiedKFold para realizar um k-fold com k=10
skf = StratifiedKFold(n_splits=10)

# conversão das listas de subjects e rótulos em numpy arrays
nbs = np.array(nbs)
labels = np.array(labels)

# listas para coleta dos subjects (nbs, de newborns) e rótulos de treinamento
treino_nbs=[]
treino_lbs=[]

# listas para coleta dos subjects (nbs, de newborns) e rótulos de teste
teste_nbs=[]
teste_lbs=[]


# A função StratifiedKFold retorna os índices para divisão dos vetores fornecidos
for train_index, test_index in skf.split(nbs, labels):
  treino_nbs.append(nbs[train_index])
  teste_nbs.append(nbs[test_index])
  treino_lbs.append(labels[train_index])
  teste_lbs.append(labels[test_index])

In [ ]:
print("Divisão dos 56 subjects por iteração da validação cruzada\n")
c=0
for bb_treino, bb_teste, lb_treino, lb_teste in zip(treino_nbs, teste_nbs, treino_lbs, teste_lbs):

  print("Fold " + str(c) + " -- Treinamento: " + str(len(bb_treino)) + " Teste: " + str(len(bb_teste)))
  c+=1


Divisão dos 56 subjects por iteração da validação cruzada

Fold 0 -- Treinamento: 50 Teste: 6
Fold 1 -- Treinamento: 50 Teste: 6
Fold 2 -- Treinamento: 50 Teste: 6
Fold 3 -- Treinamento: 50 Teste: 6
Fold 4 -- Treinamento: 50 Teste: 6
Fold 5 -- Treinamento: 50 Teste: 6
Fold 6 -- Treinamento: 51 Teste: 5
Fold 7 -- Treinamento: 51 Teste: 5
Fold 8 -- Treinamento: 51 Teste: 5
Fold 9 -- Treinamento: 51 Teste: 5


In [ ]:
print("Composição dos conjuntos de teste\n")
n=0
for lb_teste in teste_lbs:
  c = lb_teste.tolist().count('C')
  u = lb_teste.tolist().count('U')

  print("Fold " + str(n) + " -- Subjects iCOPE: " + str(c) + " || Subjects UNIFESP: " + str(u))
  n+=1

Composição dos conjuntos de teste

Fold 0 -- Subjects iCOPE: 3 || Subjects UNIFESP: 3
Fold 1 -- Subjects iCOPE: 3 || Subjects UNIFESP: 3
Fold 2 -- Subjects iCOPE: 3 || Subjects UNIFESP: 3
Fold 3 -- Subjects iCOPE: 3 || Subjects UNIFESP: 3
Fold 4 -- Subjects iCOPE: 3 || Subjects UNIFESP: 3
Fold 5 -- Subjects iCOPE: 3 || Subjects UNIFESP: 3
Fold 6 -- Subjects iCOPE: 2 || Subjects UNIFESP: 3
Fold 7 -- Subjects iCOPE: 2 || Subjects UNIFESP: 3
Fold 8 -- Subjects iCOPE: 2 || Subjects UNIFESP: 3
Fold 9 -- Subjects iCOPE: 2 || Subjects UNIFESP: 3


In [ ]:
print("Composição dos conjuntos de treinamento\n")
n=0
for lb_treino in treino_lbs:
  c = lb_treino.tolist().count('C')
  u = lb_treino.tolist().count('U')

  print("Fold " + str(n) + " -- Subjects iCOPE: " + str(c) + " || Subjects UNIFESP: " + str(u))
  n+=1

Composição dos conjuntos de treinamento

Fold 0 -- Subjects iCOPE: 23 || Subjects UNIFESP: 27
Fold 1 -- Subjects iCOPE: 23 || Subjects UNIFESP: 27
Fold 2 -- Subjects iCOPE: 23 || Subjects UNIFESP: 27
Fold 3 -- Subjects iCOPE: 23 || Subjects UNIFESP: 27
Fold 4 -- Subjects iCOPE: 23 || Subjects UNIFESP: 27
Fold 5 -- Subjects iCOPE: 23 || Subjects UNIFESP: 27
Fold 6 -- Subjects iCOPE: 24 || Subjects UNIFESP: 27
Fold 7 -- Subjects iCOPE: 24 || Subjects UNIFESP: 27
Fold 8 -- Subjects iCOPE: 24 || Subjects UNIFESP: 27
Fold 9 -- Subjects iCOPE: 24 || Subjects UNIFESP: 27


In [ ]:
# Salvando as listas com os conjuntos de treinamento e teste por iteração da
# validação cruzada. Os arquivos .npy gerados serão utilizados no script de 
# treinamento e validação dos modelos

treino_final = np.array(treino_nbs,dtype=object)
teste_final = np.array(teste_nbs,dtype=object)

np.save('NB_KFold10_Train.npy',treino_final)
np.save('NB_KFold10_Test.npy',teste_final)

# Data Augmentation + RetinaFace

Esta seção apresenta a geração de novas imagens para treinamento, por meio do Data Augmentation, e a validação dessas novas amostras pelo RetinaFace, com o objetivo de garantir a utilização exemplos nos quais a face humana é detectável.

### Data Augmentation

In [ ]:
# Configurando gerador de novas amostras

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

datagen = ImageDataGenerator(rotation_range=30, 
                             shear_range=0.15,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             brightness_range=(0.5, 1.1),
                             zoom_range=[0.7, 1.5],
                             horizontal_flip=True)

Antes da execução do script a seguir, as imagens de cada base foram organizadas em pastas de acordo com o indivíduo da amostra.

Por exemplo:

*   Base A
  *   ID_01
      * ID01_imagem_1
      * ID01_imagem_2
      * ...
      * ID01_imagem_*N*

  * ID_02
  * ID_03
  * ...
  * ID_*N*

*   Base B

In [ ]:
# Diretórios

dir_base = 'diretório das base de dados original'
dir_base_aug = 'diretório destino da base aumentada'

# criação de pastas para organizar as novas amostras por subject da base
for fold in os.listdir(dir_base):
  aug_dir = osp.join(dir_base_aug, fold + "_aug")
  os.mkdir(aug_dir)


In [ ]:
# Geração de novas amostras

NUM_DATAGEN=20  # 20 novas amostras a partir de uma imagem

# -- Data Augmentation -- #

dir_base = 'diretório das base de dados original'
dir_base_aug = 'diretório destino da base aumentada'

# varrendo indivíduos da base
for fold in tqdm(os.listdir(dir_base)):

  # varrendo imagens do indivíduo
  for img in os.listdir(osp.join(dir_base,fold)):

    # endereço da imagem
    img_path = osp.join(dir_base,fold,img)

    # carregando imagem
    image = load_img(img_path)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)

    # enderço destino das novas amostras
    new_path = osp.join(dir_base_aug, fold + "_aug")

    # preparação do gerador de imagens
    datagen.fit(image)

    # prefixo para o nome das novas amotras
    prefix = 'aug_'+img.split('.png')[0]

    # geração das novas amostras
    for x, val in zip(datagen.flow(image, save_to_dir=new_path,
                                        save_prefix=prefix, save_format='png'),range(NUM_DATAGEN)):     
      pass

print("FIM!")

### Validação - Detecção de faces

In [ ]:
#Validacao RetinaFace

dir_base_aug = 'diretório da base aumentada'
c_faces=0
c_nofaces=0

# varrendo indivíduos da base
for fold in tqdm(os.listdir(dir_base_aug)):

  # varrendo imagens "aumentadas" do indivíduo
  for img in os.listdir(osp.join(dir_base_aug,fold)):

    # endereço da imagem
    file_path = osp.join(dir_base_aug,fold,img)

    # tentativa de detecção facial
    try:
        face, _ = face_detection(file_path,retinaface)
        c_faces+=1

    # caso não a face não seja encontrada, 
    # a amostra gerada por Data Augmentation é deletada    
    except:
        os.remove(file_path)
        c_nofaces+=1
        pass

print("\nImagens aprovadas: ",c_faces)
print("Imagens rejeitas: ",c_nofaces)
print("Total: ", c_faces + c_nofaces)
print("-------------------------")

# Divisão "Dor" e "Sem_dor"

Para a execução da rotina de treinamento e validação dos modelos (apresentada no arquivo Treinamento.ipynb), a base de dados original e a base de dados aumentada devem estar organizadas por indivíduo, e as imagens de cada indívudo devem ser separadas em "Dor" e "Sem Dor":

> 
*   Base A
  *   ID_01
      * Dor
          * ID01_imagem_dor_1
          * ID01_imagem_dor_2
          * ...
          * ID01_imagem_dor_*N*
      * Sem Dor
          * ID01_imagem_semdor_1
          * ID01_imagem_semdor_2
          * ...
          * ID01_imagem_semdor_*N*
  * ID_02
      *  Dor
      *  Sem Dor
  * ID_03
  * ...
  * ID_*N*
*   Base B

In [ ]:
# Criação de diretórios conforme a organização apresentada anteriormente


# Base Original
path = 'diretorio da base original, separada por indivíduos'
path2 ='um novo diretorio para receber a base original, conforme a nova organização'

# varrendo indivíduos da base original
for fold in os.listdir(path):
  # pasta para indivíduo no novo endereço
  new_path = osp.join(path2,fold)

  # pasta "Dor" para o indivíduo no novo endereço
  new_path_dor = osp.join(new_path,"Dor")

  # pasta "Sem Dor" para o indivíduo no novo endereço
  new_path_sem_dor = osp.join(new_path,"Sem_dor")

  # criação das pastas
  os.mkdir(new_path)
  os.mkdir(new_path_dor)
  os.mkdir(new_path_sem_dor)


# Base Aumentada
path = 'diretorio da base aumentada, separada por indivíduos'
path2 ='um novo diretorio para receber a base aumentada, conforme a nova organização'

# varrendo indivíduos da base aumentada
for fold in os.listdir(path):
  # pasta para indivíduo no novo endereço
  new_path = osp.join(path2,fold)

  # pasta "Dor" para o indivíduo no novo endereço
  new_path_dor = osp.join(new_path,"Dor")

  # pasta "Sem Dor" para o indivíduo no novo endereço
  new_path_sem_dor = osp.join(new_path,"Sem_dor")

  # criação das pastas
  os.mkdir(new_path)
  os.mkdir(new_path_dor)
  os.mkdir(new_path_sem_dor)

In [ ]:
# Cópia das imagens para as novas pastas

## Base Original ##
path1 = 'diretorio da base original, separada por indivíduos'
path2 ='um novo diretorio para receber a base original, conforme a nova organização'

# varrendo indivíduos da base
for fold in os.listdir(path1):

  # varrendo imagens do indivíduo
  for img in os.listdir(osp.join(path1,fold)):

    # endereço de origem da imagem
    path_src = osp.join(path1,fold,img)

    # determinação da classe a partir do nome da imagem
    classe = "Sem_dor" if "sem_dor" in img else "Dor

    # endereço destino da imagem, considerando sua classe
    path_dest = osp.join(path2,fold,classe,img)

    # cópia da imagem no novo endereço
    copyfile(path_src,path_dest)

## Base Aumentada ##
path1 = 'diretorio da base aumentada, separada por indivíduos'
path2 ='um novo diretorio para receber a base aumentada, conforme a nova organização'

# varrendo indivíduos da base
for fold in os.listdir(path1):

  # varrendo imagens do indivíduo
  for img in os.listdir(osp.join(path1,fold)):

    # endereço de origem da imagem
    path_src = osp.join(path1,fold,img)

    # determinação da classe a partir do nome da imagem
    classe = "Sem_dor" if "sem_dor" in img else "Dor

    # endereço destino da imagem, considerando sua classe
    path_dest = osp.join(path2,fold,classe,img)

    # cópia da imagem no novo endereço
    copyfile(path_src,path_dest)